# Mask R-CNN - Test Script/ Inference Script

 

In [ ]:
import math
from PIL import Image, ImageDraw
from PIL import ImagePath
import pandas as pd
import os
from os import path
from tqdm import tqdm
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib

from google.colab.patches import cv2_imshow


from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Setting Up MaskRcnn Framework using maskrcnn implemented by matterport

src= https://github.com/matterport/Mask_RCNN

In [ ]:
%cd '/content/drive/MyDrive/mask_rcnn/Mask_RCNN'
!ls

/content/drive/MyDrive/mask_rcnn/Mask_RCNN
'=5.1.2'   dist      MANIFEST.in	  requirements.txt
 app.py    images    mask_rcnn.egg-info   samples
 assets    LICENSE   mrcnn		  setup.cfg
 build	   logs      README.md		  setup.py


In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.6.0


In [ ]:

!pip uninstall keras

Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.6.0


In [ ]:
!pwd

/content/drive/MyDrive/mask_rcnn/Mask_RCNN


In [ ]:
!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

     |████████████████████████████████| 345.0 MB 4.1 kB/s 
     |████████████████████████████████| 312 kB 46.0 MB/s 
     |████████████████████████████████| 2.9 MB 31.5 MB/s 
     |████████████████████████████████| 3.2 MB 28.5 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 367 kB 49.5 MB/s 
     |████████████████████████████████| 384 kB 43.4 MB/s 
     |████████████████████████████████| 154 kB 46.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
/usr/local/li

In [16]:
import tensorflow as tf
print(tf.__version__)


1.13.1


In [ ]:
import keras
print(keras.__version__)

2.2.3


Using TensorFlow backend.


In [ ]:
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
import numpy as np

import imutils
import cv2
import os
import time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
%matplotlib inline


In [ ]:
#installing streamlit and pyngrok
!pip -q install streamlit
!pip -q install pyngrok

     |████████████████████████████████| 8.0 MB 3.6 MB/s 
     |████████████████████████████████| 170 kB 43.7 MB/s 
     |████████████████████████████████| 75 kB 3.3 MB/s 
     |████████████████████████████████| 111 kB 48.4 MB/s 
     |████████████████████████████████| 4.3 MB 19.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 124 kB 51.0 MB/s 
     |████████████████████████████████| 787 kB 34.3 MB/s 
     |████████████████████████████████| 370 kB 35.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.27.0 which is incompatibl

In [ ]:
!pip install ipykernel>=5.1.2
!pip install pydeck

In [ ]:
from pyngrok import ngrok

!ngrok authtoken 1y63hH4HJ7IEJEmlt77c7PyKEXO_5MRrAxSpioZLs517pYK6g

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# StreamlitAPP 

In [17]:
%%writefile app.py



import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
import numpy as np
import cv2
import os
import time
from matplotlib import pyplot
import matplotlib.pyplot as plt 
 
import streamlit as st

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D
from tensorflow.keras import optimizers
       

def predict_pipeline(input_img):
  class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 101
   
    # Learning rate
    LEARNING_RATE=0.001
   
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.95
   
    # setting Max ground truth instances
    MAX_GT_INSTANCES=100

  config=myMaskRCNNConfig()


  class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
  config_ = InferenceConfig()
  #config.display()

  model_ = modellib.MaskRCNN(mode="inference", model_dir='./',
                             config=config_)
  #'/content/drive/MyDrive/mask_rcnn/
  WEIGHTS_PATH = "drive/MyDrive/mask_rcnn/v1.h5"
  WEIGHTS_PATH = "../v1.h5"


  model_.load_weights(WEIGHTS_PATH,by_name=True)
  #st.title('mode_ loaded succesfully with type {0}'.format(type(model_)))
 


  def getpred(boxes,image):
    print('input image shape is == ',image.shape)
    preds=[]
    for i in range(len(boxes)):

      box = boxes[i]
      col_s, col_e = box[1], box[3]
      row_s, row_e = box[0], box[2]
      pred_mask=image[row_s:row_e, col_s:col_e]
      print('predicted mask shape is == ',pred_mask.shape)
      preds.append(pred_mask)
    return preds

 

  def fix_dimension(img):
    new_img = np.zeros((28,28,3))
    for i in range(3):
      new_img[:,:,i] = img
    return new_img
 

  def cnnCharRecognition(img,char_model):
    dictionary={}
   
    characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i,c in enumerate(characters):
        dictionary[i] = c

    #blackAndWhiteChar=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blackAndWhiteChar = cv2.resize(img,(28,28))
    blackAndWhiteChar=fix_dimension(blackAndWhiteChar)
    image = blackAndWhiteChar.reshape((1, 28,28, 3))
    image = image / 255.0
    new_predictions = char_model.predict(image)
    char = np.argmax(new_predictions)
    return dictionary[char]
  def extract_char(charlist,char_model):

    ans=[]
    for i in range(len(charlist)):
      #plt.subplot(1, 12, i+1)
      z=cnnCharRecognition(charlist[i],char_model)
      #print('pred-{0} is {1}'.format(i,z))
      ans.append(z)
      
    #plt.title('char== {0}'.format( z))

    #plt.imshow(charlist[i], cmap='gray')
    #plt.axis('off')
    print('opencv+ yolo == ',''.join(ans))
    return ans

  # Find characters in the resulting images
  def segment_characters_(image) :

    img = cv2.resize(image, (333, 75))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   
   
    _, img_binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
   
    img_erode = cv2.erode(img_binary, (3,3))
   
   
   
    img_dilate = cv2.dilate(img_erode, (3,3))

    LP_WIDTH = img_dilate.shape[0]
    LP_HEIGHT = img_dilate.shape[1]


    # Make borders white
    img_dilate[0:5,:] = 255
    img_dilate[:,0:5] = 255
    img_dilate[72:75,:] = 255
    img_dilate[:,330:333] = 255
   
    dimensions = [5, LP_HEIGHT/9, 0.27*LP_WIDTH, .8*LP_WIDTH]

    # Get contours within cropped license plate
    char_list,ii = find_contours_(dimensions, img_dilate)
    print('no of characters found are = ',len(char_list))
    return char_list,ii

 
  
  # Find characters in the resulting images
  def segment_characters_(image) :
    img = cv2.resize(image, (333, 75))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   
    
    _, img_binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    img_erode = cv2.erode(img_binary, (3,3))
    
    
    
    img_dilate = cv2.dilate(img_erode, (3,3))

    LP_WIDTH = img_dilate.shape[0]
    LP_HEIGHT = img_dilate.shape[1]


    # Make borders white
    img_dilate[0:5,:] = 255
    img_dilate[:,0:5] = 255
    img_dilate[72:75,:] = 255
    img_dilate[:,330:333] = 255
   
    dimensions = [5, LP_HEIGHT/9, 0.27*LP_WIDTH, .8*LP_WIDTH]

    # Get contours within cropped license plate
    char_list,ii = find_contours_(dimensions, img_dilate)
    print('no of characters found are = ',len(char_list))
    return char_list,ii

  # Match contours to license plate or character template
  def find_contours_(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
     
    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
     
    ii=img.copy()
     

    x_cntr_list = []
    target_contours = []
    img_res = []
    cnt=0
    for i,cntr in enumerate(cntrs) :
      #detects contour in binary image and returns the coordinates of rectangle enclosing it
      intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
         
        
      #checking the dimensions of the contour to filter out the characters by contour's size
      if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
        cnt+=1   
            
        x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

        char_copy = np.zeros((44,24))
        #extracting each character using the enclosing rectangle's coordinates.
        char = img[intY:intY+intHeight, intX:intX+intWidth]
        char = cv2.resize(char, (20, 40))
            
            
        cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
             

             
        #Make result formatted for classification: invert colors
        char = cv2.subtract(255, char)

        # Resize the image to 24x44 with black border
        char_copy[2:42, 2:22] = char
        char_copy[0:2, :] = 0
        char_copy[:, 0:2] = 0
        char_copy[42:44, :] = 0
        char_copy[:, 22:24] = 0
        if cnt>10:
          break
        img_res.append(char_copy) #List that stores the character's binary image (unsorted)
            
    #Return characters on ascending order with respect to the x-coordinate (most-left character first)
  
    #function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list[:10])), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
      img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res,ii


    

  def load_char_cnn():

    model = Sequential()
    model.add(Conv2D(32, (24,24), input_shape=(28, 28, 3), activation='relu', padding='same'))
    # model.add(Conv2D(32, (20,20), input_shape=(28, 28, 3), activation='relu', padding='same'))
    # model.add(Conv2D(32, (20,20), input_shape=(28, 28, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(36, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00001), metrics=['accuracy'])
    model.summary()
    #loading pretrained weights saved using training char cnn
    #model.load_weights('drive/MyDrive/mask_rcnn/m1.h5')
    model.load_weights('../m1.h5')
    return model

  char_model=load_char_cnn()
  #print(char_model.summary)

  print('-------'*5)
  results = model_.detect([input_img], verbose=1)
  print('hi')
  print(results[0].keys())
  print(results[0]['rois'].shape)
  print('hii')
  #ndarray
  if results[0]['rois'].shape[0]>0:


    # no of paltes found are
    print(results[0]['rois'].shape[0])
    #boudning box coordinate are
    print(results[0]['rois'])
    #confidence probability is
    print(results[0]['scores'])
    #predicted class id us 
    print(results[0]['class_ids'])
    mask_=np.uint8(results[0]['masks'][:,:,0])
     
    masked = cv2.bitwise_and(input_img, input_img, mask=mask_)

    # prediction imgut__
    pred_plate=getpred(list(results[0]['rois']),input_img)

    
    plt.figure(figsize=(10,6))
    
    plt.subplot(141)
    plt.title('Input img ')
    plt.imshow(input_img)
 
    
    plt.subplot(142)
    #mask_=np.uint8(results[0]['masks'][:,:,0])
    #masked = cv2.bitwise_and(input_img, input_img, mask=mask_)
    plt.title("predicted mask")
    plt.imshow(masked)


    plt.subplot(143)
    plt.imshow(pred_plate[0])
    plt.title('predicted plate')
    #st.image(pred_plate[0], caption='predicted plate')
    #st.title('plate shape is {0} '.format(pred_plate[0].shape))
    
    charlist,ii=segment_characters_(pred_plate[0])
    plt.subplot(144)
     
    
    plt.title('contours extracted')
    
    plt.imshow(ii)
    #st.image(ii, caption='contours extracted')
    #st.title('contour shape is {0} '.format(ii.shape))

    plt.show()
    #segmenting characters
    #charlist,ii=segment_characters_(pred_plate[0])
    number_plate_is=extract_char(charlist,char_model)

    #print('extracted license plate content is == ',''.join(number_plate_is))
    print('******'*5)
    plt.subplot(143)
    plt.imshow(pred_plate[0])
    plt.title('predicted plate')

   
    number_plate_is=extract_char(charlist,char_model)

   
   
  #plt.show()
  print('extracted license plate content is == ',''.join(number_plate_is))
  print('******'*5)
  return ''.join(number_plate_is),input_img,masked,pred_plate[0],ii

st.title("Extract License Plate from Car Images")

file_upload= st.file_uploader(label = "Upload your image here",type=['png','jpeg','jpg'])
submit_button_object = st.button('Submit once uploaded')

if submit_button_object:
  if file_upload is not None:
    try:

      #file_details = {"FileName":file_upload.name,"FileType":file_upload.type,"FileSize":file_upload.size}
      #st.write(file_details)
      #st.title('file uploaded name is {0} '.format(type(file_upload)))
      data = file_upload.read()

      input_image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)

    

      #st.title('file uploaded type is {0} '.format(type(data)))
      #st.title('file uploaded type is {0} '.format(type(input_image)))
    
      st.write('shape of input_img {0} '.format(input_image.shape))
      st.write('Please wait for some time, your file is being processed')
      #st.title("yaha tak pahuch gya")
      ans,input_img,masked,pred_plate,ii= predict_pipeline(input_image) 
      

      if ans:
        st.subheader("Extracted License plate is == {0} ".format(ans))
        st.write("Results")
        st.image(input_img,width=300 ,caption='Input image')

        st.image(masked,width=250, caption='predicted mask')
        st.image(pred_plate, caption='predicted plate')
        st.image(ii, caption='contours extracted')

    
      else:
        st.subheader("Please try with other image ")
    except:
      st.write("Please upload file with correct format ")

  else:
    st.subheader('Please upload an image in correct format')
  
 

st.caption("Made  by github@jakashcode")
  

Overwriting app.py


In [18]:
!ls

'=5.1.2'   dist      MANIFEST.in	  requirements.txt
 app.py    images    mask_rcnn.egg-info   samples
 assets    LICENSE   mrcnn		  setup.cfg
 build	   logs      README.md		  setup.py


In [19]:
public_url = ngrok.connect(port='80')
print (public_url)
 

NgrokTunnel: "http://17b4-34-79-162-208.ngrok.io" -> "http://localhost:80"


In [20]:
!streamlit run --server.port 80 app.py >/dev/null

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
!ls

app.py	drive  img1.jpg  sample_data
